<a href="https://colab.research.google.com/github/FaQ2108/Trading-Algoritmico-con-SmallCaps/blob/main/Estrategia2_CrucedeMedias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estrategia de Cruce de Medias en Trading Algorítmico

La **estrategia de cruce de medias** es una técnica sencilla y popular en trading que se basa en la intersección de dos medias móviles para identificar posibles cambios de tendencia en el mercado. A continuación, se explica de manera clara cómo funciona y cómo se aplica en el trading algorítmico.

## ¿Qué es la Estrategia de Cruce de Medias?

- **Medias Móviles:**  
  Se calculan dos tipos de medias móviles (pueden ser simples o exponenciales) sobre los datos de precios:
  - **Media Móvil de Corto Plazo:** Es más sensible a los cambios recientes en el precio.
  - **Media Móvil de Largo Plazo:** Suaviza las fluctuaciones y refleja la tendencia general.

- **Cruce de Medias:**  
  La estrategia se basa en la intersección (o cruce) de estas dos medias:
  - **Cruce Alcista (Señal de Compra):** Ocurre cuando la media de corto plazo cruza por encima de la media de largo plazo, sugiriendo que la tendencia podría estar cambiando hacia un movimiento alcista.
  - **Cruce Bajista (Señal de Venta):** Se produce cuando la media de corto plazo cruza por debajo de la media de largo plazo, indicando una posible reversión a una tendencia bajista.

## Señales de Trading

- **Señal de Compra:**  
  Cuando la media móvil de corto plazo supera a la de largo plazo, se interpreta como un indicativo de que el impulso positivo está creciendo.

- **Señal de Venta:**  
  Cuando la media móvil de corto plazo cae por debajo de la de largo plazo, se considera una señal de que el impulso negativo está tomando control.

## Aplicación en Trading Algorítmico

En el trading algorítmico, esta estrategia se implementa mediante la programación de reglas específicas que se ejecutan automáticamente:

1. **Cálculo de Medias Móviles:**  
   El algoritmo obtiene los datos históricos o en tiempo real y calcula las medias móviles de corto y largo plazo según los períodos establecidos.

2. **Monitoreo de Cruces:**  
   Se verifica continuamente si ocurre un cruce entre ambas medias. Al detectar un cruce alcista o bajista, se genera la señal correspondiente.

3. **Ejecución Automática:**  
   Con la señal detectada, el sistema puede ejecutar automáticamente órdenes de compra o venta según las reglas predefinidas.

## Consideraciones Importantes

- **Retraso en la Señal:**  
  Al utilizar promedios, la estrategia puede presentar un retraso, ya que las medias móviles responden de forma más lenta a los cambios bruscos en el precio.

- **Optimización de Parámetros:**  
  Es fundamental elegir períodos adecuados para las medias móviles, ya que estos parámetros afectan la sensibilidad de la estrategia y la cantidad de señales generadas.

- **Uso de Filtros Adicionales:**  
  Se recomienda combinar la estrategia de cruce de medias con otros indicadores o herramientas (como el RSI o el MACD) para confirmar las señales y minimizar el riesgo de operaciones falsas.

- **Backtesting:**  
  Antes de implementar la estrategia en un entorno real, es esencial realizar pruebas con datos históricos para evaluar su desempeño y ajustar parámetros según sea necesario.

---

En resumen, la estrategia de cruce de medias en trading algorítmico utiliza la intersección entre una media móvil de corto plazo y una de largo plazo para generar señales de compra o venta, permitiendo automatizar la toma de decisiones y seguir tendencias de mercado de manera objetiva.

In [ ]:
!pip install pandas sqlalchemy pymysql h5py google-colab
!pip install pandas_ta
!pip install tables
!pip install --upgrade tables

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 918.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=840d1c69c20acf904ff04ced6b8231b8cea9c346ded7beecbaa1afa6f4442b1e
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas_ta


In [ ]:
from getpass import getpass  # Importar getpass para ocultar la entrada de la contraseña
import pandas as pd
import sqlalchemy as db
from sqlalchemy.exc import SQLAlchemyError
import h5py
from google.colab import drive
import os

class DatabaseAnalyzer:
    def __init__(self):
        self.engine = None
        self.cleaned_dataframes = {}

    def get_db_connection(self):
        try:
            host = input("Introduce el host de la base de datos: ")
            user = input("Introduce el usuario de la base de datos: ")
            password = getpass("Introduce la contraseña de la base de datos: ")  # Oculta la contraseña
            database = input("Introduce el nombre de la base de datos: ")

            connection_string = f"mysql+pymysql://{user}:{password}@{host}/{database}"
            self.engine = db.create_engine(connection_string)
            print("Conexión establecida correctamente.")
            self.analyze_tables()
        except SQLAlchemyError as e:
            print(f"Error al conectar a la base de datos: {e}")

    def count_nulls(self, df):
        return df.isnull().sum()

    def analyze_tables(self):
        try:
            # Proceso de limpieza para la tabla vista_datos_completos
            query_vista = "SELECT * FROM vista_datos_completos"
            df_vista = pd.read_sql(query_vista, self.engine)
            print("Valores nulos antes de la limpieza (vista_datos_completos):")
            print(self.count_nulls(df_vista))

            columns_to_check = df_vista.columns.difference(['spread', 'short_float'])
            df_vista_cleaned = df_vista.dropna(subset=columns_to_check, how='any')

            print("Valores nulos después de la limpieza (vista_datos_completos):")
            print(self.count_nulls(df_vista_cleaned))

            # Aplicar el filtro adicional a vista_datos_completos
            df_vista_cleaned = self.filter_vista_datos_completos(df_vista_cleaned)
            # **Filtro nuevo:** Filtrar por horarios entre 15:30 y 22:00 usando la columna 'fecha'
            df_vista_cleaned = self.filter_by_time(df_vista_cleaned, 'fecha')

            # Proceso de limpieza para la tabla OHLCData
            query_ohlc = "SELECT * FROM OHLCData"
            df_ohlc = pd.read_sql(query_ohlc, self.engine)
            print("Valores nulos antes de la limpieza (OHLCData):")
            print(self.count_nulls(df_ohlc))

            # Limpiar nulos en OHLCData
            df_ohlc_cleaned = df_ohlc.dropna()
            # **Filtro nuevo:** Filtrar por horarios entre 15:30 y 22:00 usando la columna 'date'
            df_ohlc_cleaned = self.filter_by_time(df_ohlc_cleaned, 'date')

            print("Valores nulos después de la limpieza (OHLCData):")
            print(self.count_nulls(df_ohlc_cleaned))

            # Obtener los id_event que cumplen con las condiciones en ambas tablas
            cleaned_events_vista = df_vista_cleaned['id_event'].unique()
            cleaned_events_ohlc = df_ohlc_cleaned['id_event'].unique()

            # Encontrar la intersección de id_event entre ambas tablas
            common_events = set(cleaned_events_vista).intersection(set(cleaned_events_ohlc))

            # Filtrar ambas tablas para mantener solo los registros con id_event comunes
            df_vista_final = df_vista_cleaned[df_vista_cleaned['id_event'].isin(common_events)]
            df_ohlc_final = df_ohlc_cleaned[df_ohlc_cleaned['id_event'].isin(common_events)]

            # Se guardan ambas tablas: vista_datos_completos y OHLCData limpias.
            self.cleaned_dataframes['vista_datos_completos'] = df_vista_final
            self.cleaned_dataframes['OHLCData'] = df_ohlc_final

            print("Tablas vista_datos_completos y OHLCData filtradas y limpias guardadas correctamente.")
        except Exception as e:
            print(f"Error al analizar las tablas: {e}")

    def filter_vista_datos_completos(self, df):
        """
        Filtra la tabla vista_datos_completos según las condiciones:
        - percent_var_max < 500
        - percent_var_min > -40
        - ratio_vol < 1500
        - float_shares < 3000000
        - market_cap < 300000000
        - precio < 10
        """
        try:
            required_columns = ['precio', 'float_shares', 'market_cap']
            if all(column in df.columns for column in required_columns):
                filtered_df = df[(df['percent_var'] < 500) & (df['percent_var'] > -40) &
                                 (df['ratio_vol'] < 1500) &
                                 (df['float_shares'] < 3000000) &
                                 (df['market_cap'] < 300000000) &
                                 (df['precio'] < 10)]
                print("vista_datos_completos filtrada según las condiciones especificadas.")
                return filtered_df
            else:
                print("Advertencia: No se encontraron todas las columnas requeridas para el filtrado.")
                return df
        except Exception as e:
            print(f"Error al filtrar vista_datos_completos: {e}")
            return df

    def filter_by_time(self, df, time_column):
        """
        Filtra un DataFrame para incluir solo registros cuyo valor en la columna de tiempo esté entre las 15:30 y las 22:00.
        Se asume que la columna indicada contiene valores que pueden convertirse a datetime.
        """
        try:
            if time_column not in df.columns:
                print(f"Columna '{time_column}' no encontrada en el DataFrame. Se omite el filtro por tiempo.")
                return df

            # Convertir la columna a datetime (en caso de que no lo sea)
            df.loc[:, time_column] = pd.to_datetime(df[time_column], errors='coerce')

            # Definir los límites de hora
            hora_inicio = pd.to_datetime("15:30").time()
            hora_fin = pd.to_datetime("22:00").time()
            # Filtrar el DataFrame
            filtered_df = df[(df[time_column].dt.time >= hora_inicio) & (df[time_column].dt.time <= hora_fin)]
            print(f"Filtrado por horario entre 15:30 y 22:00 en la columna '{time_column}'.")
            return filtered_df
        except Exception as e:
            print(f"Error al filtrar por horario: {e}")
            return df

    def save_cleaned_data_to_hdf5(self, file_path):
        try:
            if self.cleaned_dataframes:
                # Verificar si el archivo ya existe
                if os.path.exists(file_path):
                    overwrite = input(f"El archivo {file_path} ya existe. ¿Desea sobrescribirlo? (s/n): ").strip().lower()
                    if overwrite != 's':
                        new_file_path = input("Introduce la nueva ruta del archivo HDF5: ")
                        file_path = new_file_path

                # Forzar el cierre del archivo si está abierto
                try:
                    import tables
                    if tables.is_hdf5_file(file_path):
                        with pd.HDFStore(file_path, mode='r') as store:
                            store.close()
                except Exception:
                    pass

                # Guardar los DataFrames en el archivo HDF5
                with pd.HDFStore(file_path, mode='w') as store:
                    for table_name, df in self.cleaned_dataframes.items():
                        store.put(table_name, df)
                        print(f"Tabla '{table_name}' guardada en {file_path}")
            else:
                print("No hay datos limpios para guardar.")
        except Exception as e:
            print(f"Error al guardar los DataFrames en HDF5: {e}")


# Montar Google Drive (si ya está montado, este mensaje se puede omitir o forzarse el remount con force_remount=True)
drive.mount('/content/drive')

# Crear una instancia de DatabaseAnalyzer
analyzer = DatabaseAnalyzer()

# Establecer la conexión a la base de datos
analyzer.get_db_connection()

# Guardar los DataFrames limpios y filtrados en un archivo HDF5
file_path = '/content/drive/MyDrive/Proyecto_SmallCaps/BBDD/bbdd_filtrada.h5'
analyzer.save_cleaned_data_to_hdf5(file_path)

Mounted at /content/drive
Introduce el host de la base de datos: librobot.org
Introduce el usuario de la base de datos: faq2108
Introduce la contraseña de la base de datos: ··········
Introduce el nombre de la base de datos: smallcaps
Conexión establecida correctamente.
Valores nulos antes de la limpieza (vista_datos_completos):
ticker            0
id_event          0
fecha             0
percent_var       0
ratio_vol         0
precio            3
volumen           0
spread          128
insercion         0
anterior          0
apertura          0
float_shares    416
exchange        231
country         231
avg_volume      231
shs_outstand    245
market_cap      233
inst_own        231
short_float     426
dtype: int64
Valores nulos después de la limpieza (vista_datos_completos):
ticker            0
id_event          0
fecha             0
percent_var       0
ratio_vol         0
precio            0
volumen           0
spread          111
insercion         0
anterior          0
apertura      

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

# Ruta al archivo HDF5 generado previamente
data_path = "/content/drive/MyDrive/Proyecto_SmallCaps/BBDD/bbdd_filtrada.h5"

# Cargar datos desde la tabla 'OHLCData'
with pd.HDFStore(data_path, "r") as store:
    df = store["OHLCData"] # NOMBRE DE LA TABLA A UTILIZAR

# Filtrar datos: descartamos registros con volumen 0
df = df[df["volume"] > 0].reset_index(drop=True)
print("Datos cargados:", df.shape)

# Convertir la columna 'date' a datetime, si aún no lo está

df["date"] = pd.to_datetime(df["date"]) #En OHLCData la columna se llama 'date' mientras que en vista_datos_completos y en tabla_fusionada, se llama 'fecha'.

df

Datos cargados: (27835, 9)


,id_ohlc,id_event,date,open,high,low,close,volume,synced
0,4100,14,2025-01-15 15:30:00,5.280,5.280,5.280,5.280,6002,0
1,4101,14,2025-01-15 15:38:00,5.410,5.410,5.410,5.410,500,0
2,4102,14,2025-01-15 15:41:00,5.410,5.410,5.410,5.410,1000,0
3,4103,14,2025-01-15 15:46:00,5.320,5.336,5.320,5.336,785,0
4,4104,14,2025-01-15 15:52:00,5.392,5.392,5.392,5.392,300,0
...,...,...,...,...,...,...,...,...,...
27830,894852,3512,2025-02-28 21:56:00,3.310,3.310,3.260,3.260,7438,0
27831,894853,3512,2025-02-28 21:57:00,3.230,3.260,3.230,3.250,4223,0
27832,894854,3512,2025-02-28 21:58:00,3.250,3.339,3.250,3.331,31823,0
27833,894855,3512,2025-02-28 21:59:00,3.320,3.360,3.250,3.330,37352,0


In [ ]:
def aplicar_cruce_medias(df_event, ema_rapida=10, ema_lenta=50, min_candles=30):
    """
    Aplica la estrategia de cruce de medias móviles a un DataFrame correspondiente a un evento.
    Se calcula:
      - EMA rápida sobre el precio de cierre.
      - EMA lenta sobre el precio de cierre.
      - Señal de compra: cuando la EMA rápida cruza por encima de la lenta.
      - Señal de venta: cuando la EMA rápida cruza por debajo de la lenta.

    Parámetros:
      - df_event: DataFrame de un evento (debe tener las columnas date, open, high, low, close).
      - ema_rapida: Período para la EMA rápida.
      - ema_lenta: Período para la EMA lenta.
      - min_candles: Número mínimo de velas para considerar el evento.

    Retorna:
      - DataFrame procesado con columnas adicionales: EMA_rapida, EMA_lenta, Compra, Venta.
      - Si el evento tiene menos de min_candles, retorna None.
    """
    if len(df_event) < min_candles:
        return None

    df_event = df_event.sort_values("date").reset_index(drop=True)

    # Calcular EMAs
    df_event["EMA_rapida"] = df_event["close"].ewm(span=ema_rapida, adjust=False).mean()
    df_event["EMA_lenta"] = df_event["close"].ewm(span=ema_lenta, adjust=False).mean()

    # Señal de Compra: cuando la EMA rápida cruza de abajo hacia arriba la EMA lenta
    df_event["Compra"] = (
        (df_event["EMA_rapida"] > df_event["EMA_lenta"]) &
        (df_event["EMA_rapida"].shift(1) <= df_event["EMA_lenta"].shift(1))
    )

    # Señal de Venta: cuando la EMA rápida cruza de arriba hacia abajo la EMA lenta
    df_event["Venta"] = (
        (df_event["EMA_rapida"] < df_event["EMA_lenta"]) &
        (df_event["EMA_rapida"].shift(1) >= df_event["EMA_lenta"].shift(1))
    )

    return df_event

In [ ]:
# Diccionario para almacenar el DataFrame procesado de cada evento
resultados = {}

# Parámetros de la estrategia (ajustables según el comportamiento de smallcaps)
ema_rapida = 10
ema_lenta = 50
min_candles = 30  # Mínimo de velas para considerar un evento

# Asegurarse de que la columna 'id_event' exista
if "id_event" not in df.columns:
    raise Exception("La columna 'id_event' no se encontró en el DataFrame.")

# Procesar cada evento por separado
event_ids = df["id_event"].unique()
print("Eventos encontrados:", len(event_ids))

for event in event_ids:
    df_event = df[df["id_event"] == event].copy()
    df_event_proc = aplicar_cruce_medias(df_event, ema_rapida, ema_lenta, min_candles)
    if df_event_proc is not None:
        resultados[event] = df_event_proc

print("Eventos procesados (con datos suficientes):", len(resultados))


Eventos encontrados: 195
Eventos procesados (con datos suficientes): 145


In [ ]:
def calcular_pnl_evento(df_event, stop_loss_pct=5.0):
    """
    Calcula el PnL de un evento basándose en:
      - Primer 'Compra' como entrada (usamos el precio de cierre de esa vela).
      - Primera 'Venta' posterior como salida (o el stop loss si se activa antes).
      - Si no hay 'Venta', salimos en la última vela.
      - El stop loss se activa si el 'low' de alguna vela cae stop_loss_pct% por debajo del precio de entrada.
    """
    df_event = df_event.sort_values("date").reset_index(drop=True)

    # 1) Buscar el índice de la primera señal de Compra
    compra_indices = df_event.index[df_event["Compra"] == True].tolist()
    if len(compra_indices) == 0:
        # No hay señal de compra -> no hacemos nada (None o retornamos info nula)
        return None

    buy_idx = compra_indices[0]  # Tomamos la primera señal de compra
    entry_price = df_event.loc[buy_idx, "close"]  # Usamos el cierre de la vela de compra
    entry_date  = df_event.loc[buy_idx, "date"]

    # 2) A partir de la vela siguiente a la compra, buscamos la salida
    exit_price = None
    exit_date = None
    motivo = ""

    # Para no revisar velas anteriores a la compra, iteramos desde buy_idx+1
    for idx in range(buy_idx+1, len(df_event)):
        row = df_event.iloc[idx]

        # Verificar stop loss
        # Si el low de esta vela cae un X% (stop_loss_pct) por debajo del precio de entrada
        if row["low"] <= entry_price * (1 - stop_loss_pct/100.0):
            exit_price = entry_price * (1 - stop_loss_pct/100.0)
            exit_date  = row["date"]
            motivo = "Stop Loss"
            break

        # Verificar la señal de Venta
        if row["Venta"] == True:
            # Podemos usar el precio de cierre de la vela
            exit_price = row["close"]
            exit_date  = row["date"]
            motivo = "Venta (Cruce de Medias)"
            break

    # Si no encontramos ninguna salida en el bucle anterior,
    # usamos la última vela del DataFrame
    if exit_price is None:
        exit_price = df_event.iloc[-1]["close"]
        exit_date  = df_event.iloc[-1]["date"]
        motivo = "Fin del Evento (sin Venta)"

    # 3) Calcular PnL
    pnl_pct = ((exit_price - entry_price) / entry_price) * 100.0

    # 4) Retornamos un diccionario con toda la info
    return {
        "entry_date": entry_date,
        "entry_price": entry_price,
        "exit_date": exit_date,
        "exit_price": exit_price,
        "motivo_exit": motivo,
        "pnl_pct": pnl_pct
    }

In [ ]:
resumen_operaciones = []

for event_id, df_event in resultados.items():
    resultado = calcular_pnl_evento(df_event, stop_loss_pct=5.0)
    if resultado is not None:
        # Añadimos el id_event
        resultado["id_event"] = event_id
        resumen_operaciones.append(resultado)

df_operaciones = pd.DataFrame(resumen_operaciones)
print(df_operaciones.head())

           entry_date  entry_price           exit_date  exit_price  \
0 2025-01-15 15:38:00        5.410 2025-01-15 16:01:00      5.1395   
1 2025-01-15 15:31:00        2.015 2025-01-15 15:58:00      2.1300   
2 2025-01-15 17:19:00        4.000 2025-01-15 18:16:00      4.1750   
3 2025-01-16 15:58:00        7.270 2025-01-16 20:36:00     10.3680   
4 2025-01-16 15:52:00        3.920 2025-01-16 15:58:00      3.7830   

               motivo_exit    pnl_pct  id_event  
0                Stop Loss  -5.000000        14  
1  Venta (Cruce de Medias)   5.707196        19  
2  Venta (Cruce de Medias)   4.375000        31  
3  Venta (Cruce de Medias)  42.613480       248  
4  Venta (Cruce de Medias)  -3.494898       249  


In [ ]:
# Filtrar solo los eventos con PnL positivo
df_positivos = df_operaciones[df_operaciones["pnl_pct"] > 0]

# Seleccionar el Top 10 de mayor PnL (orden descendente)
df_positivos_top10 = df_positivos.sort_values("pnl_pct", ascending=False).head(10)

print("Top 10 eventos con mayor PnL positivo:")
print(df_positivos_top10)

Top 10 eventos con mayor PnL positivo:
             entry_date  entry_price           exit_date  exit_price  \
22  2025-01-23 15:32:00        2.880 2025-01-23 20:04:00      21.710   
35  2025-01-27 15:33:00        2.710 2025-01-27 18:02:00       6.810   
45  2025-01-30 15:31:00        3.104 2025-01-30 17:10:00       7.450   
3   2025-01-16 15:58:00        7.270 2025-01-16 20:36:00      10.368   
125 2025-02-25 15:32:00        3.465 2025-02-25 16:28:00       4.870   
79  2025-02-06 15:31:00        0.138 2025-02-06 17:37:00       0.181   
60  2025-02-03 15:39:00        0.540 2025-02-03 17:40:00       0.707   
139 2025-02-28 15:33:00        0.920 2025-02-28 18:36:00       1.190   
104 2025-02-20 15:47:00        5.475 2025-02-20 17:12:00       6.933   
19  2025-01-22 15:34:00        0.870 2025-01-22 22:00:00       1.090   

                    motivo_exit     pnl_pct  id_event  
22      Venta (Cruce de Medias)  653.819444       742  
35      Venta (Cruce de Medias)  151.291513       965  


In [ ]:
def calcular_comision(cantidad_acciones, precio_accion):
    valor_negociado = cantidad_acciones * precio_accion

    if valor_negociado <= 300_000:
        tasa = 0.0035
    elif valor_negociado <= 3_000_000:
        tasa = 0.0020
    elif valor_negociado <= 20_000_000:
        tasa = 0.0015
    elif valor_negociado <= 100_000_000:
        tasa = 0.0010
    else:
        tasa = 0.0005

    comision = valor_negociado * tasa
    comision = max(comision, 0.35)
    comision = min(comision, 0.01 * valor_negociado)
    return comision

def calcular_gestion_riesgo(equity, precio_compra, porcentaje_stop_loss, porcentaje_riesgo):
    """
    Calcula la posición óptima basada en riesgo, incluyendo comisiones.
    Parámetros:
        equity (float): Capital total disponible.
        precio_compra (float): Precio de compra por acción.
        porcentaje_stop_loss (float): % de stop loss (ej: 0.05 = 5%).
        porcentaje_riesgo (float): % de capital a arriesgar (ej: 0.01 = 1%).

    Devuelve:
        dict: Detalles de la operación.
    """
    if porcentaje_stop_loss >= 1 or porcentaje_stop_loss <= 0:
        raise ValueError("El porcentaje de stop loss debe ser un valor entre 0 y 1")
    if porcentaje_riesgo >= 1 or porcentaje_riesgo <= 0:
        raise ValueError("El riesgo debe ser un valor entre 0 y 1")

    stop_loss = precio_compra * (1 - porcentaje_stop_loss)
    riesgo_por_accion = precio_compra - stop_loss
    monto_arriesgar = equity * porcentaje_riesgo

    # Búsqueda binaria para determinar la cantidad óptima de acciones
    low, high = 1, int(equity // precio_compra)
    mejor_cantidad = 0
    mejor_comision = 0

    while low <= high:
        mid = (low + high) // 2
        comision = calcular_comision(mid, precio_compra)
        costo_total = mid * precio_compra + comision
        riesgo_total = mid * riesgo_por_accion + comision

        if riesgo_total <= monto_arriesgar and costo_total <= equity:
            mejor_cantidad = mid
            mejor_comision = comision
            low = mid + 1
        else:
            high = mid - 1

    costo_total = mejor_cantidad * precio_compra + mejor_comision
    porcentaje_portafolio = (costo_total / equity) * 100

    return {
        'Acciones a comprar': mejor_cantidad,
        'Costo total posición': round(costo_total, 2),
        'Comisión': round(mejor_comision, 2),
        '% del portafolio': round(porcentaje_portafolio, 2),
        'Riesgo por acción': round(riesgo_por_accion, 2),
        'Monto a arriesgar': round(monto_arriesgar, 2),
        'Stop Loss': round(stop_loss, 2),
        'Relación Riesgo/Posición': f"{round((monto_arriesgar/costo_total)*100, 1)}%" if costo_total > 0 else "0%"
    }

In [ ]:
# Lista para acumular información de comisiones en cada trade
lista_comisiones = []

for event in df_positivos_top10["id_event"]:
    # Obtener el DataFrame del evento y ordenarlo por fecha
    df_plot = resultados[event].sort_values("date").reset_index(drop=True)

    # Extraer la información de la operación para este evento
    op = df_positivos_top10[df_positivos_top10["id_event"] == event].iloc[0]
    entry_date = op["entry_date"]
    exit_date  = op["exit_date"]
    entry_price = op["entry_price"]
    exit_price  = op["exit_price"]
    pnl_pct     = op["pnl_pct"]

    # ------------------------------------------------
    # Integrar la gestión de riesgo y cálculo de comisión
    # ------------------------------------------------
    capital = 500.00
    porcentaje_stop_loss = 0.05  # 5%
    porcentaje_riesgo = 0.01     # 1%

    gestion = calcular_gestion_riesgo(
        equity=capital,
        precio_compra=entry_price,  # Se utiliza el precio de entrada; puedes ajustarlo según la orden
        porcentaje_stop_loss=porcentaje_stop_loss,
        porcentaje_riesgo=porcentaje_riesgo
    )

    # Acumular la comisión para el resumen global
    lista_comisiones.append(gestion['Comisión'])

    # Mostrar la información de gestión de riesgo en consola
    print(f"\n=== Gestión de Riesgo para Evento {event} ===")
    print(f"Capital disponible: €{capital:.2f}")
    print(f"Precio de compra: €{entry_price:.2f}")
    print(f"% Stop loss: {porcentaje_stop_loss * 100}%")
    print(f"% Riesgo: {porcentaje_riesgo * 100}%")
    for key, value in gestion.items():
        print(f"{key}: {value}")

    # ------------------------------------------------
    # Filtrado por fecha y restricción horaria
    # ------------------------------------------------
    day_of_entry = entry_date.date()
    df_plot = df_plot[df_plot["date"].dt.date == day_of_entry]
    df_plot = df_plot[
        (
            (df_plot["date"].dt.hour > 15) |
            ((df_plot["date"].dt.hour == 15) & (df_plot["date"].dt.minute >= 30))
        ) &
        (
            (df_plot["date"].dt.hour < 22) |
            ((df_plot["date"].dt.hour == 22) & (df_plot["date"].dt.minute == 0))
        )
    ]

    if df_plot.empty:
        print(f"Evento {event} no tiene datos en el rango especificado.")
        continue

    # ------------------------------------------------
    # Crear el gráfico Candlestick con Plotly
    # ------------------------------------------------
    fig = go.Figure()

    # Añadir velas (OHLC)
    fig.add_trace(go.Candlestick(
        x=df_plot["date"],
        open=df_plot["open"],
        high=df_plot["high"],
        low=df_plot["low"],
        close=df_plot["close"],
        name="OHLC"
    ))

    # Añadir EMAs para visualizar los cruces
    fig.add_trace(go.Scatter(
        x=df_plot["date"],
        y=df_plot["EMA_rapida"],
        mode="lines",
        name="EMA Rápida",
        line=dict(color="orange", dash="dot")
    ))
    fig.add_trace(go.Scatter(
        x=df_plot["date"],
        y=df_plot["EMA_lenta"],
        mode="lines",
        name="EMA Lenta",
        line=dict(color="purple", dash="dot")
    ))

    # Marcar punto de entrada
    fig.add_trace(go.Scatter(
        x=[entry_date],
        y=[entry_price],
        mode="markers",
        marker=dict(color="green", size=10, symbol="triangle-up"),
        name="Entrada"
    ))

    # Marcar punto de salida
    fig.add_trace(go.Scatter(
        x=[exit_date],
        y=[exit_price],
        mode="markers",
        marker=dict(color="red", size=10, symbol="triangle-down"),
        name="Salida"
    ))

    # Configurar layout del gráfico
    fig.update_layout(
        title=f"Evento {event} - PnL: {pnl_pct:.2f}%",
        xaxis_title="Fecha",
        yaxis_title="Precio",
        xaxis_rangeslider_visible=False  # Se oculta el slider inferior
    )

    # Mostrar el gráfico
    fig.show()

# ------------------------------------------------
# Resumen global de comisiones
total_comisiones = sum(lista_comisiones)
media_comision = total_comisiones / len(lista_comisiones) if lista_comisiones else 0
print("\n=== Resumen Global de Comisiones ===")
print(f"Total de comisiones pagadas: €{total_comisiones:.2f}")
print(f"Comisión promedio: €{media_comision:.2f}")


=== Gestión de Riesgo para Evento 742 ===
Capital disponible: €500.00
Precio de compra: €2.88
% Stop loss: 5.0%
% Riesgo: 1.0%
Acciones a comprar: 32
Costo total posición: 92.51
Comisión: 0.35
% del portafolio: 18.5
Riesgo por acción: 0.14
Monto a arriesgar: 5.0
Stop Loss: 2.74
Relación Riesgo/Posición: 5.4%



=== Gestión de Riesgo para Evento 965 ===
Capital disponible: €500.00
Precio de compra: €2.71
% Stop loss: 5.0%
% Riesgo: 1.0%
Acciones a comprar: 34
Costo total posición: 92.49
Comisión: 0.35
% del portafolio: 18.5
Riesgo por acción: 0.14
Monto a arriesgar: 5.0
Stop Loss: 2.57
Relación Riesgo/Posición: 5.4%



=== Gestión de Riesgo para Evento 1346 ===
Capital disponible: €500.00
Precio de compra: €3.10
% Stop loss: 5.0%
% Riesgo: 1.0%
Acciones a comprar: 29
Costo total posición: 90.37
Comisión: 0.35
% del portafolio: 18.07
Riesgo por acción: 0.16
Monto a arriesgar: 5.0
Stop Loss: 2.95
Relación Riesgo/Posición: 5.5%



=== Gestión de Riesgo para Evento 248 ===
Capital disponible: €500.00
Precio de compra: €7.27
% Stop loss: 5.0%
% Riesgo: 1.0%
Acciones a comprar: 12
Costo total posición: 87.59
Comisión: 0.35
% del portafolio: 17.52
Riesgo por acción: 0.36
Monto a arriesgar: 5.0
Stop Loss: 6.91
Relación Riesgo/Posición: 5.7%



=== Gestión de Riesgo para Evento 3323 ===
Capital disponible: €500.00
Precio de compra: €3.46
% Stop loss: 5.0%
% Riesgo: 1.0%
Acciones a comprar: 26
Costo total posición: 90.44
Comisión: 0.35
% del portafolio: 18.09
Riesgo por acción: 0.17
Monto a arriesgar: 5.0
Stop Loss: 3.29
Relación Riesgo/Posición: 5.5%


...


=== Gestión de Riesgo para Evento 1591 ===
Capital disponible: €500.00
Precio de compra: €0.54
% Stop loss: 5.0%
% Riesgo: 1.0%
Acciones a comprar: 172
Costo total posición: 93.23
Comisión: 0.35
% del portafolio: 18.65
Riesgo por acción: 0.03
Monto a arriesgar: 5.0
Stop Loss: 0.51
Relación Riesgo/Posición: 5.4%



=== Gestión de Riesgo para Evento 3510 ===
Capital disponible: €500.00
Precio de compra: €0.92
% Stop loss: 5.0%
% Riesgo: 1.0%
Acciones a comprar: 101
Costo total posición: 93.27
Comisión: 0.35
% del portafolio: 18.65
Riesgo por acción: 0.05
Monto a arriesgar: 5.0
Stop Loss: 0.87
Relación Riesgo/Posición: 5.4%



=== Gestión de Riesgo para Evento 2962 ===
Capital disponible: €500.00
Precio de compra: €5.47
% Stop loss: 5.0%
% Riesgo: 1.0%
Acciones a comprar: 16
Costo total posición: 87.95
Comisión: 0.35
% del portafolio: 17.59
Riesgo por acción: 0.27
Monto a arriesgar: 5.0
Stop Loss: 5.2
Relación Riesgo/Posición: 5.7%



=== Gestión de Riesgo para Evento 603 ===
Capital disponible: €500.00
Precio de compra: €0.87
% Stop loss: 5.0%
% Riesgo: 1.0%
Acciones a comprar: 106
Costo total posición: 92.57
Comisión: 0.35
% del portafolio: 18.51
Riesgo por acción: 0.04
Monto a arriesgar: 5.0
Stop Loss: 0.83
Relación Riesgo/Posición: 5.4%



=== Resumen Global de Comisiones ===
Total de comisiones pagadas: €3.50
Comisión promedio: €0.35


In [ ]:
import numpy as np
import pandas as pd

# Dividir los eventos en Entrenamiento (60%), Validación (20%) y Test (20%)
unique_events = df["id_event"].unique()
np.random.shuffle(unique_events)  # Mezclamos aleatoriamente
n_total = len(unique_events)
n_train = int(0.6 * n_total)
n_val = int(0.2 * n_total)
# El resto serán para test
n_test = n_total - n_train - n_val

train_events = unique_events[:n_train]
val_events   = unique_events[n_train:n_train+n_val]
test_events  = unique_events[n_train+n_val:]

print(f"Total de eventos: {n_total}")
print(f"Entrenamiento (60%): {len(train_events)}")
print(f"Validación (20%): {len(val_events)}")
print(f"Test (20%): {len(test_events)}")

# Los mejores parámetros encontrados en el estudio con Optuna:
best_params = {
    'ema_rapida': 6,
    'ema_lenta': 55,
    'min_candles': 22,
    'stop_loss_pct': 2.6182843610736852
}

# Función para procesar un conjunto de eventos usando los parámetros óptimos
def procesar_conjunto(events_list, df, best_params):
    resultados_subset = {}
    for event in events_list:
        df_event = df[df["id_event"] == event].copy()
        df_event_proc = aplicar_cruce_medias(df_event,
                                             ema_rapida=best_params["ema_rapida"],
                                             ema_lenta=best_params["ema_lenta"],
                                             min_candles=best_params["min_candles"])
        if df_event_proc is not None:
            resultados_subset[event] = df_event_proc

    resumen_operaciones = []
    for event_id, df_event_proc in resultados_subset.items():
        op = calcular_pnl_evento(df_event_proc, stop_loss_pct=best_params["stop_loss_pct"])
        if op is not None:
            op["id_event"] = event_id
            resumen_operaciones.append(op)
    return pd.DataFrame(resumen_operaciones)

# Procesar cada subconjunto
df_operaciones_train = procesar_conjunto(train_events, df, best_params)
df_operaciones_val   = procesar_conjunto(val_events, df, best_params)
df_operaciones_test  = procesar_conjunto(test_events, df, best_params)

# Función para calcular métricas sobre un conjunto de operaciones
def calcular_metricas(df_operaciones, events_list, df):
    total_eventos = len(events_list)
    total_trades  = len(df_operaciones)
    winning_trades = df_operaciones[df_operaciones["pnl_pct"] > 0]
    losing_trades  = df_operaciones[df_operaciones["pnl_pct"] < 0]
    n_winning = len(winning_trades)
    n_losing  = len(losing_trades)
    porc_winning = (n_winning / total_trades * 100) if total_trades > 0 else 0
    porc_losing  = (n_losing / total_trades * 100) if total_trades > 0 else 0
    net_profit_pct = df_operaciones["pnl_pct"].sum()
    avg_pnl = df_operaciones["pnl_pct"].mean() if total_trades > 0 else 0
    sum_wins = winning_trades["pnl_pct"].sum() if n_winning > 0 else 0
    sum_losses = losing_trades["pnl_pct"].sum() if n_losing > 0 else 0
    profit_factor = (sum_wins / abs(sum_losses)) if sum_losses != 0 else np.nan
    win_loss_ratio = (n_winning / n_losing) if n_losing > 0 else np.nan
    moda_pnl = df_operaciones["pnl_pct"].mode().iloc[0] if not df_operaciones["pnl_pct"].mode().empty else 0
    avg_loss = losing_trades["pnl_pct"].mean() if n_losing > 0 else 0
    dias_analizados = df["date"].dt.date[df["id_event"].isin(events_list)].nunique()
    # Se asume que el importe mínimo por trade es el 5% del precio de entrada
    importe_minimo = (df_operaciones["entry_price"] * 0.05).mean() if total_trades > 0 else 0
    if total_trades > 0:
        var95 = np.percentile(df_operaciones["pnl_pct"], 5)
        es95 = df_operaciones[df_operaciones["pnl_pct"] <= var95]["pnl_pct"].mean() if not df_operaciones[df_operaciones["pnl_pct"] <= var95].empty else var95
        var99 = np.percentile(df_operaciones["pnl_pct"], 1)
        es99 = df_operaciones[df_operaciones["pnl_pct"] <= var99]["pnl_pct"].mean() if not df_operaciones[df_operaciones["pnl_pct"] <= var99].empty else var99
    else:
        var95 = es95 = var99 = es99 = 0
    return {
        "total_eventos": total_eventos,
        "total_trades": total_trades,
        "n_winning": n_winning,
        "porc_winning": porc_winning,
        "n_losing": n_losing,
        "porc_losing": porc_losing,
        "net_profit_pct": net_profit_pct,
        "avg_pnl": avg_pnl,
        "profit_factor": profit_factor,
        "win_loss_ratio": win_loss_ratio,
        "moda_pnl": moda_pnl,
        "avg_loss": avg_loss,
        "dias_analizados": dias_analizados,
        "importe_minimo": importe_minimo,
        "var95": var95,
        "es95": es95,
        "var99": var99,
        "es99": es99
    }

# Calcular métricas para cada conjunto
metricas_train = calcular_metricas(df_operaciones_train, train_events, df)
metricas_val   = calcular_metricas(df_operaciones_val, val_events, df)
metricas_test  = calcular_metricas(df_operaciones_test, test_events, df)

# Mostrar resultados
print("=== Resultados del Conjunto de Entrenamiento ===")
for key, value in metricas_train.items():
    print(f"{key}: {value}")

print("\n=== Resultados del Conjunto de Validación ===")
for key, value in metricas_val.items():
    print(f"{key}: {value}")

print("\n=== Resultados del Conjunto de Test ===")
for key, value in metricas_test.items():
    print(f"{key}: {value}")

Total de eventos: 195
Entrenamiento (60%): 117
Validación (20%): 39
Test (20%): 39
=== Resultados del Conjunto de Entrenamiento ===
total_eventos: 117
total_trades: 92
n_winning: 27
porc_winning: 29.347826086956523
n_losing: 64
porc_losing: 69.56521739130434
net_profit_pct: 261.0781533633183
avg_pnl: 2.837806014818677
profit_factor: 2.726536945925264
win_loss_ratio: 0.421875
moda_pnl: -2.6182843610736883
avg_loss: -2.362733190233409
dias_analizados: 26
importe_minimo: 0.13007826086956523
var95: -2.6182843610736923
es95: -2.6182843610736946
var99: -2.6182843610736954
es99: -2.618284361073696

=== Resultados del Conjunto de Validación ===
total_eventos: 39
total_trades: 26
n_winning: 11
porc_winning: 42.30769230769231
n_losing: 15
porc_losing: 57.692307692307686
net_profit_pct: 114.34349742726687
avg_pnl: 4.397826824125649
profit_factor: 4.208674358438626
win_loss_ratio: 0.7333333333333333
moda_pnl: -2.6182843610736892
avg_loss: -2.3757162544203587
dias_analizados: 19
importe_minimo: 0.1

In [ ]:
!pip install optuna

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import optuna

# --- Supongamos que ya están definidas las funciones:
# def aplicar_cruce_medias(df_event, ema_rapida, ema_lenta, min_candles):
#     # ... implementación ...
#     return df_event_procesado
#
# def calcular_pnl_evento(df_event, stop_loss_pct):
#     # ... implementación ...
#     return resultado_evento  # Diccionario con, al menos, la clave "pnl_pct"

# Ejemplo:
# df ya está cargado y preparado, y train_events contiene los id_event del 60% de la bbdd.
# Por ejemplo:
# train_events = unique_events[:n_train]

def objective_with_cv(trial):
    # Sugerir parámetros a optimizar
    ema_rapida = trial.suggest_int("ema_rapida", 5, 30)
    ema_lenta = trial.suggest_int("ema_lenta", 30, 120)
    # Penalizar si ema_lenta no es mayor que ema_rapida
    if ema_lenta <= ema_rapida:
        return -1000.0
    min_candles = trial.suggest_int("min_candles", 20, 40)
    stop_loss_pct = trial.suggest_float("stop_loss_pct", 2.0, 10.0)

    # Usar KFold para validación cruzada (5 folds)
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    net_profits = []

    # Aquí train_events es un array/lista de id_event correspondientes al conjunto de entrenamiento.
    for train_idx, val_idx in kf.split(train_events):
        # Para cada fold, usaremos los eventos de validación (val_events_fold)
        val_events_fold = [train_events[i] for i in val_idx]

        resultados_val = {}
        for event in val_events_fold:
            df_event = df[df["id_event"] == event].copy()
            df_event_proc = aplicar_cruce_medias(df_event,
                                                 ema_rapida=ema_rapida,
                                                 ema_lenta=ema_lenta,
                                                 min_candles=min_candles)
            if df_event_proc is not None:
                resultados_val[event] = df_event_proc

        resumen_operaciones = []
        for event_id, df_event_proc in resultados_val.items():
            op = calcular_pnl_evento(df_event_proc, stop_loss_pct=stop_loss_pct)
            if op is not None:
                resumen_operaciones.append(op)
        df_operaciones = pd.DataFrame(resumen_operaciones)

        if df_operaciones.empty:
            net_profits.append(-1000.0)
        else:
            # Sumar el pnl% de los trades del fold
            net_profits.append(df_operaciones["pnl_pct"].sum())

    # Retornar el promedio del net profit obtenido en los 5 folds
    return np.mean(net_profits)

# Crear y ejecutar el estudio con validación cruzada
study = optuna.create_study(direction="maximize")
study.optimize(objective_with_cv, n_trials=100, n_jobs=4)

print("Mejores parámetros encontrados:")
print(study.best_params)
print("Mejor valor del objetivo (Net Profit promedio en CV):", study.best_value)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 4.7 MB/s eta 0:00:00


[I 2025-03-13 14:57:59,463] A new study created in memory with name: no-name-96b0733a-0831-4527-8128-479a0867910a
[I 2025-03-13 14:58:05,257] Trial 3 finished with value: 181.13537770210607 and parameters: {'ema_rapida': 6, 'ema_lenta': 54, 'min_candles': 21, 'stop_loss_pct': 4.170842357472495}. Best is trial 3 with value: 181.13537770210607.
[I 2025-03-13 14:58:05,514] Trial 2 finished with value: 166.4366151493702 and parameters: {'ema_rapida': 29, 'ema_lenta': 33, 'min_candles': 36, 'stop_loss_pct': 6.390188828670588}. Best is trial 3 with value: 181.13537770210607.
[I 2025-03-13 14:58:05,927] Trial 1 finished with value: 181.55245334739453 and parameters: {'ema_rapida': 20, 'ema_lenta': 49, 'min_candles': 27, 'stop_loss_pct': 7.40635789151268}. Best is trial 1 with value: 181.55245334739453.
[I 2025-03-13 14:58:06,012] Trial 0 finished with value: 170.46976035107917 and parameters: {'ema_rapida': 15, 'ema_lenta': 64, 'min_candles': 25, 'stop_loss_pct': 6.656183380171384}. Best is t

Mejores parámetros encontrados:
{'ema_rapida': 25, 'ema_lenta': 67, 'min_candles': 29, 'stop_loss_pct': 4.981761289243441}
Mejor valor del objetivo (Net Profit promedio en CV): 206.68777687468537


# Walk-Forward Validation para la Estrategia de Cruce de Medias

A continuación se muestra cómo aplicar un esquema de validación cruzada temporal (walk-forward) a la estrategia de cruce de medias. Este método respeta el orden cronológico de los eventos y permite evaluar la robustez de la estrategia en distintos periodos.

## 1. Preparación de los Eventos de Entrenamiento

Primero, extraemos los eventos del conjunto de entrenamiento y los ordenamos por la fecha de inicio (por ejemplo, la fecha mínima de cada evento). Esto es fundamental para realizar una validación temporal.

In [ ]:
event_dates_cruce = {}
for event in train_events:
    df_event = df[df["id_event"] == event]
    event_dates_cruce[event] = df_event["date"].min()

# Ordenar los eventos de entrenamiento por su fecha de inicio
sorted_train_events_cruce = sorted(train_events, key=lambda x: event_dates_cruce[x])


In [ ]:
def procesar_conjunto_cruce(events_list, df, best_params):
    resultados_subset = {}
    for event in events_list:
        df_event = df[df["id_event"] == event].copy()
        df_event_proc = aplicar_cruce_medias(
            df_event,
            ema_rapida=best_params["ema_rapida"],
            ema_lenta=best_params["ema_lenta"],
            min_candles=best_params["min_candles"]
        )
        if df_event_proc is not None:
            resultados_subset[event] = df_event_proc

    resumen_operaciones = []
    for event_id, df_event_proc in resultados_subset.items():
        op = calcular_pnl_evento(df_event_proc, stop_loss_pct=best_params["stop_loss_pct"])
        if op is not None:
            op["id_event"] = event_id
            resumen_operaciones.append(op)
    return pd.DataFrame(resumen_operaciones)

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
import numpy as np

# Configurar el TimeSeriesSplit con 5 folds
tscv = TimeSeriesSplit(n_splits=5)
fold_performances_cruce = []

# Supongamos que best_params_cruce contiene los parámetros óptimos para la estrategia de cruce de medias.
# Por ejemplo, estos podrían haber sido obtenidos en una fase de optimización previa:
best_params_cruce = {
    "ema_rapida": 25,
    "ema_lenta": 67,
    "min_candles": 29,
    "stop_loss_pct": 4.981761289243441
}

for fold, (train_idx, test_idx) in enumerate(tscv.split(sorted_train_events_cruce)):
    # En cada split, usamos los eventos del fold como conjunto de prueba
    test_fold = [sorted_train_events_cruce[i] for i in test_idx]

    # Procesar los eventos del fold con la estrategia de cruce de medias
    df_operaciones_fold = procesar_conjunto_cruce(test_fold, df, best_params_cruce)

    # Calcular el Net Profit (suma de pnl_pct) para este fold
    if df_operaciones_fold.empty:
        fold_profit = -1000.0  # Penalización si no se generan operaciones
    else:
        fold_profit = df_operaciones_fold["pnl_pct"].sum()

    print(f"Fold {fold+1}: Net Profit = {fold_profit:.2f}%")
    fold_performances_cruce.append(fold_profit)

# Calcular el rendimiento promedio en todos los folds
avg_performance_cruce = np.mean(fold_performances_cruce)
print("Walk-Forward Validation (Cruce de Medias): Average Net Profit =", avg_performance_cruce)

Fold 1: Net Profit = 242.02%
Fold 2: Net Profit = 24.87%
Fold 3: Net Profit = -2.73%
Fold 4: Net Profit = 4.14%
Fold 5: Net Profit = 52.95%
Walk-Forward Validation (Cruce de Medias): Average Net Profit = 64.25201799942477


# Ajuste de Net Profit por Comisiones en la Validación Walk-Forward

## Supuestos

- **Comisión por trade:** 0.05 USD al entrar y 0.05 USD al salir, es decir, **0.10 USD por operación completa**.
- **Capital inicial:** €500.
- **Capital utilizado por operación:** 10% del capital inicial, es decir, **€50 por trade**.
- **Número de trades:** Se estima un número promedio de trades por fold (por ejemplo, 20 trades).

## Cálculo del Impacto de las Comisiones

1. **Costo Total de Comisiones por Fold:**

   \[
   \text{Costo\_Total\_Comisiones} = \text{Número\_de\_Trades} \times 0.10\ \text{USD}
   \]
   
   Ejemplo: Si se tienen 20 trades, entonces:
   
   \[
   20 \times 0.10 = 2.00\ \text{USD}
   \]

2. **Impacto en Porcentaje sobre el Capital Inicial:**

   \[
   \text{Impacto\_Comisiones (\%)} = \left(\frac{\text{Costo\_Total\_Comisiones}}{\text{Capital\_Inicial}}\right) \times 100
   \]
   
   Con un capital de €500:
   
   \[
   \frac{2.00}{500} \times 100 = 0.40\%
   \]

3. **Cálculo del Net Profit Ajustado:**

   Se resta el impacto de las comisiones al Net Profit original de cada fold:
   
   \[
   \text{Net Profit Ajustado} = \text{Net Profit Original} - \text{Impacto Comisiones (\%)}
   \]

   **Ejemplo de Tabla:**

   

   El **Net Profit Promedio Ajustado** se calcula como el promedio de estos valores:
   
   \[
   \text{Promedio} \approx \frac{31.19 + 59.69 + 33.19 + 32.20 - 34.00}{5} \approx 24.45\%
   \]

## Ejemplo de Código

A continuación, un fragmento de código en Python que ilustra cómo hacer este ajuste:

In [ ]:
# Supuestos
comision_por_trade = 0.05  # USD por trade (entrada o salida)
comision_operacion = 2 * comision_por_trade  # 0.10 USD por operación completa
capital_inicial = 500.00  # Capital en euros

# Ejemplo: Para un fold con net profit original y un número estimado de trades
net_profit_original = 64.25  # En porcentaje
numero_trades = 20

# Calcular el costo total de comisiones en USD
costo_total_comisiones = numero_trades * comision_operacion  # Ej: 20 * 0.10 = 2.00 USD

# Calcular el impacto de las comisiones en porcentaje sobre el capital inicial
impacto_comisiones_pct = (costo_total_comisiones / capital_inicial) * 100  # (2.00 / 500) * 100 = 0.40%

# Calcular el Net Profit Ajustado
net_profit_ajustado = net_profit_original - impacto_comisiones_pct

print(f"Net Profit Original: {net_profit_original:.2f}%")
print(f"Impacto de Comisiones: {impacto_comisiones_pct:.2f}%")
print(f"Net Profit Ajustado: {net_profit_ajustado:.2f}%")


Net Profit Original: 64.25%
Impacto de Comisiones: 0.40%
Net Profit Ajustado: 63.85%
